Uploaded for the sake of uploading

In [10]:
import pandas as pd
import pyarrow

file_path_matthias = '../../Samsung_USB/'
file_path_irem = ''
file_path_silas = ''
file_path_rosa = ''
file_path_koen = '../Desktop/Data_Matrixian/'

filenames = [
    'bag_ids_no_funda',
    'random_online_sample',
    'special_house_types_class_sample',
    'detailed_woning_type_sample',
    'not_online_listings', # Download from slack
]

file_path = file_path_koen # Interchangable

In [11]:
for filename in filenames:
    filename = filename + '.parquet' if not filename.endswith('.parquet') else filename	
    df = pd.read_parquet(file_path + filename, engine='pyarrow')
    df.to_csv(
        file_path + filename.replace('.parquet', '.csv'),
        index=False,
        encoding='utf-8',
    )

In [12]:
print(filenames)

# When printing the content of the CSV files, the file size becomes too large to be pushed to GitHub.

# for filename in filenames:
#     filename = filename + '.csv' if not filename.endswith('.csv') else filename
#     print(f"Reading file: {filename}")
#     with open(file_path + filename, 'r', encoding='utf-8') as file:
#         content = file.read()
#         print(content)

['bag_ids_no_funda', 'random_online_sample', 'special_house_types_class_sample', 'detailed_woning_type_sample', 'not_online_listings']


In [13]:
import pandas as pd

df = pd.read_csv(file_path + "bag_image_summary.csv")
df_sample = pd.read_parquet(file_path + "detailed_woning_type_sample.parquet")
df = pd.read_csv(file_path + "bag_image_summary.csv", dtype="string")
df_joined = pd.merge(df_sample, df, how="left", right_on="bag_id", left_on="bag_nummeraanduidingid")
df_sample_with_urls = df_joined[df_joined["frontview_exists"].notna()]

print(df_sample_with_urls)

     bag_nummeraanduidingid  source_data_result_id  \
0          0375200000007496               43958963   
1          0241200000062495               88540113   
2          0202200000330373               43850283   
3          0797200000023888               43417804   
4          0453200000023782               43625881   
...                     ...                    ...   
7189       1719200000010702               89088335   
7190       0058200000221359               88798822   
7191       0988200000040147               43869164   
7192       0034200000041264               43870484   
7193       0230200000006335               42390765   

                        special_house_type           woningtype  \
0     Eengezinswoning, 2-onder-1-kapwoning  2-onder-1-kapwoning   
1     Eengezinswoning, 2-onder-1-kapwoning  2-onder-1-kapwoning   
2     Eengezinswoning, 2-onder-1-kapwoning  2-onder-1-kapwoning   
3     Eengezinswoning, 2-onder-1-kapwoning  2-onder-1-kapwoning   
4     Eengezinsw